In [4]:
# importing packages
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from numpy.linalg import cond, matrix_rank, norm
from scipy.linalg import inv, solve, det, eig, lu, eigvals
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm

In [5]:
#importing data
BL = pd.read_csv('Data/BankLend.csv')
consumption = pd.read_csv('Data/consumption.csv')
gdp_exp = pd.read_csv('Data/gdp.csv')
housing_starts = pd.read_csv('Data/HousingStarts.csv')
ex_rate = pd.read_csv('Data/CEER_MONTHLY_NOMINAL-sd-1990-01-01.csv') #Canadian Effective Exchange Rate - monthly nominal
SP_tbills = pd.read_csv('Data/s&p, tbills.csv')
house_prices = pd.read_csv('Data/OECD Data.csv')

In [6]:
# BANK LENDING

# Summing over rows of 3 to get quarterly data
def sum_quarter(group):
    return pd.Series({'ReferencePeriod': group['ReferencePeriod'].iloc[0],
                      'Total, funds advanced, residential mortgages, insured 7': group['Total, funds advanced, residential mortgages, insured 7'].sum(),
                      'Total, funds advanced, residential mortgages, uninsured 7': group['Total, funds advanced, residential mortgages, uninsured 7'].sum(),
                      'Total, funds advanced, variable rate mortgages': group['Total, funds advanced, variable rate mortgages'].sum(),
                      'Total, funds advanced for non-mortgage loans, consumer credit': group['Total, funds advanced for non-mortgage loans, consumer credit'].sum(),
                      'Total, funds advanced, business loans 8': group['Total, funds advanced, business loans 8'].sum()})

# Group by sets of 3 rows and average using the custom function
BankLend = BL.groupby(BL.index // 3).apply(sum_quarter)
BankLend['ReferencePeriod'] = pd.to_datetime(BankLend['ReferencePeriod'])
BankLend['Quarters'] = BankLend['ReferencePeriod'].dt.to_period("Q").astype(str)

# cleaning up the data
BankLend['BankLending'] = BankLend['Total, funds advanced, residential mortgages, insured 7'] + BankLend['Total, funds advanced, residential mortgages, uninsured 7'] +  BankLend['Total, funds advanced, variable rate mortgages'] + BankLend['Total, funds advanced for non-mortgage loans, consumer credit'] + BankLend['Total, funds advanced, business loans 8'] + BankLend['Total, funds advanced, business loans 8']
BankLend1 = BankLend[['Quarters','BankLending']]
BankLend1.head()

,Quarters,BankLending
0,2013Q1,382924.0
1,2013Q2,493784.0
2,2013Q3,483018.0
3,2013Q4,503383.0
4,2014Q1,420139.0


In [7]:
# GDP 

# keeping specific columns
gdp = gdp_exp[['Quarters', 'Gross domestic product at market prices']]

gdp.head()

,Quarters,Gross domestic product at market prices
0,2000Q1,"1,514,676"
1,2000Q2,"1,532,503"
2,2000Q3,"1,548,212"
3,2000Q4,"1,551,274"
4,2001Q1,"1,560,006"


In [8]:
# EXCHANGE RATE

# keeping one column
filtered_ex_rate = ex_rate[['date','CEER_BROADNM']]
filtered_ceer = filtered_ex_rate.reset_index(drop=True)

# Averaging over rows of 3 to get quarterly data
def average_quarter(group):
    return pd.Series({'date': group['date'].iloc[0],
                      'CEER_BROADNM': group['CEER_BROADNM'].mean()})

# Group by sets of 3 rows and average using the custom function
ceer = filtered_ceer.groupby(filtered_ceer.index // 3).apply(average_quarter).reset_index(drop=True)

# converting date to datetime format
ceer['date'] = pd.to_datetime(ceer['date'])
ceer['Quarters'] = ceer['date'].dt.to_period("Q").astype(str)

ceer = ceer.drop(['date'], axis=1)
ceer.head()

,CEER_BROADNM,Quarters
0,98.053333,1999Q1
1,101.366667,1999Q2
2,100.096667,1999Q3
3,100.486667,1999Q4
4,102.433333,2000Q1


In [9]:
# CONSUMPTION, HOUSE PRICES, HOUSING STARTS

print(consumption.head())
print(house_prices.head())
print(housing_starts.head())

  Quarters  Household final consumption expenditure
0   2000Q1                                   143884
1   2000Q2                                   145903
2   2000Q3                                   148957
3   2000Q4                                   150284
4   2001Q1                                   151594
  Quarters  House_Prices
0   2001Q1     40.844646
1   2001Q2     41.200980
2   2001Q3     41.875943
3   2001Q4     42.620664
4   2002Q1     43.340875
  Quarters HousingStarts
0   2016Q1        38,175
1   2016Q2        53,286
2   2016Q3        54,899
3   2016Q4        51,555
4   2017Q1        42,862


In [10]:
# FINANCIAL INDICATORS: S&P 500, T-BILLS

# Averaging over rows of 3 to get quarterly data
def average_quarter(group):
    return pd.Series({'nummonth': group['nummonth'].iloc[0],
                      'sp': group['sp'].mean(), 't_bills': group['t_bills'].mean()})

# Group by sets of 3 rows and average using the custom function
fin_ind = SP_tbills.groupby(SP_tbills.index // 3).apply(average_quarter).reset_index(drop=True)
fin_ind.rename(columns={'nummonth': 'Quarters'}, inplace=True)
fin_ind.head()

,Quarters,sp,t_bills
0,2013Q1,36782.440000,0.953333
1,2013Q2,36089.766667,1.010000
2,2013Q3,37059.500000,0.990000
3,2013Q4,39750.513333,0.916667
4,2014Q1,41900.943333,0.876667


In [11]:
merge1 = pd.merge(gdp, consumption, on = 'Quarters')
merge2 = pd.merge(merge1, BankLend1, on = 'Quarters')
merge3 = pd.merge(merge2, ceer, on = 'Quarters')
merge4 = pd.merge(merge3, house_prices, on = 'Quarters')
merge5 = pd.merge(merge4, housing_starts, on = 'Quarters')
final_df = pd.merge(merge5, fin_ind, on = 'Quarters')
final_df.head()
final_df.dtypes

Quarters                                    object
Gross domestic product at market prices     object
Household final consumption expenditure      int64
BankLending                                float64
CEER_BROADNM                               float64
House_Prices                               float64
HousingStarts                               object
sp                                         float64
t_bills                                    float64
dtype: object

In [12]:
final_df['Quarters'] = pd.to_datetime(final_df['Quarters'])
final_df.head()


C:\Users\chany\AppData\Local\Temp\ipykernel_20812\1545389824.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  final_df['Quarters'] = pd.to_datetime(final_df['Quarters'])


,Quarters,Gross domestic product at market prices,Household final consumption expenditure,BankLending,CEER_BROADNM,House_Prices,HousingStarts,sp,t_bills
0,2016-01-01,"2,071,732",285296,495823.0,111.046667,105.128205,"38,175",41244.963333,0.460000
1,2016-04-01,"2,061,412",286760,579799.0,117.166667,108.601056,"53,286",44654.566667,0.523333
2,2016-07-01,"2,082,805",289033,586583.0,116.170000,112.826169,"54,899",46931.136667,0.516667
3,2016-10-01,"2,094,490",292579,575781.0,115.810000,115.803167,"51,555",48612.290000,0.493333
4,2017-01-01,"2,120,843",296967,539456.0,117.380000,119.193062,"42,862",50214.126667,0.486667


In [ ]:
import numpy as np
from statsmodels.tsa.vector_ar.svar_model import SVAR

# Assuming your data is in a DataFrame named 'final_df'
# If not, replace 'final_df' with your actual DataFrame

# Convert DataFrame to NumPy array
endog_data = final_df[['Gross domestic product at market prices', 'Household final consumption expenditure', 'BankLending', 'CEER_BROADNM', 'House_Prices', 'HousingStarts', 'sp', 't_bills']].to_numpy()

# Impose Short-Term Restrictions for SVAR
B_matrix = np.array([
    [1, 'E', 'E', 0, 0, 0, 0, 0],
    [0, 1, 0, 'E', 'E', 'E', 0, 'E'],
    [0, 0, 1, 'E', 0,'E', 0, 'E'],
    [0, 0, 0, 1, 'E', 'E', 0, 'E'],
    [0, 0, 0, 0, 1, 'E', 'E', 'E'],
    [0, 0, 0, 0, 0, 1, 'E', 'E'],
    [0, 0, 0, 0, 0, 0, 1, 'E'],
    [0, 0, 0, 0, 0, 0, 0, 1],
])

# Define SVAR Model
svar_model = SVAR(endog=endog_data, svar_type='B', A=None, B=B_matrix, dates=final_df['Quarters'])


results = svar_model.fit(maxlags=4)
results['coefs'] = results['coefs'].dot(B_matrix)

# Print Identified Coefficients
identified_results = results.coefs
print(identified_results)


In [35]:
import numpy as np
from statsmodels.tsa.vector_ar.svar_model import SVAR

# Assuming your data is in a DataFrame named 'final_df'
# If not, replace 'final_df' with your actual DataFrame

# Convert DataFrame to NumPy array
endog_data = final_df[['Gross domestic product at market prices', 'Household final consumption expenditure', 'BankLending', 'CEER_BROADNM', 'House_Prices', 'HousingStarts', 'sp', 't_bills']].to_numpy()

# Impose Short-Term Restrictions for SVAR
B_matrix = np.array([
    [1, 'E', 'E', 0, 0, 0, 0, 0],
    [0, 1, 0, 'E', 'E', 'E', 0, 'E'],
    [0, 0, 1, 'E', 0,'E', 0, 'E'],
    [0, 0, 0, 1, 'E', 'E', 0, 'E'],
    [0, 0, 0, 0, 1, 'E', 'E', 'E'],
    [0, 0, 0, 0, 0, 1, 'E', 'E'],
    [0, 0, 0, 0, 0, 0, 1, 'E'],
    [0, 0, 0, 0, 0, 0, 0, 1],
])

# Define SVAR Model
svar_model = SVAR(endog=endog_data, svar_type='B', A=None, B=B_matrix, dates=final_df['Quarters'])


C:\Users\chany\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency QS-OCT will be used.
  self._init_dates(dates, freq)


In [39]:
res = svar_model.fit(maxlags=4)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
 res = svar_model.fit(maxlags=4)
# Plot Impulse Response Function
res.irf(periods=30).plot(impulse='Gross domestic product at market prices', plot_stderr=False)
plt.show()